<a href="https://colab.research.google.com/github/65487-cmyk/ML-on-financial-inclusion-in-east-africa/blob/main/kiberaprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv(r"/Editedsolar_suitability_kibera.csv")

df.drop(["irradkimax","irradtype","densitycode","potentialdemand","new AREA_s","building_1",
         "densitytyp","Unnamed: 20","Unnamed: 21","osm_type","building","builtupden"],axis=1, inplace=True)
#replace nulls
df=df.fillna(0)


In [ ]:
# conditions based on building function
conditions=[
    df["cleanfunct"]=='residential',
    df["cleanfunct"]=='institution',
    df["cleanfunct"]=='commercial'
    ]


In [ ]:
#occupancy values
density=[4,
    df["new AREA"]/5,
    df["new AREA"]/5
                ]
df["occupancy"]=np.select(conditions,density, default=0)
df

,fid,full_id,osm_id,cleanfunct,new AREA,usableroof,irradkimea,irradkW_da,iradkW_yea,energydemand/year,occupancy
0,1,w48161236,48161236,institution,575.204489,402.643142,3.655858,294.400,80592.0,120792.942600,115.040898
1,2,w169277664,169277664,commercial,205.700463,113.135255,0.314682,7.120,1949.0,22627.050980,41.140093
2,3,w397432293,397432293,commercial,233.390448,128.364747,0.720228,18.490,5062.0,25672.949330,46.678090
3,4,w397432307,397432307,commercial,62.405759,34.323168,0.519003,3.563,975.0,6864.633541,12.481152
4,5,w397432308,397432308,commercial,218.180774,119.999426,3.100813,74.410,20372.0,23999.885110,43.636155
...,...,...,...,...,...,...,...,...,...,...,...
3270,3271,w1433369270,1433369270,residential,29.049313,11.619725,0.307315,0.714,195.0,1161.972504,4.000000
3271,3272,w1433369271,1433369271,residential,34.133387,13.653355,0.307315,0.839,230.0,1365.335480,4.000000
3272,3273,w1433369272,1433369272,residential,44.327119,17.730848,3.839100,13.610,3727.0,1773.084772,4.000000
3273,3274,w1433369273,1433369273,commercial,199.909285,109.950107,0.307315,6.758,1850.0,21990.021380,39.981857


In [ ]:
# simulate the approximate daily energy demand of each building
#simulate data data
days=pd.date_range(start='2024-01-01', end='2025-01-01', freq="h")

days_df=pd.DataFrame({"date":days})
days_df["hour"] = days_df["date"].dt.hour
days_df["weekday"]=days_df["date"].dt.day_name()
days_df["isweekend"]=days_df["date"].dt.day_of_week>5


days_df["date"] = days_df["date"].dt.strftime("%d/%m/%Y %H:%M:%S")







In [ ]:
#weekday and weekend factor
weekday_factor=[0.95,1.00,1.10]
weekend_factor=[1.05,1.00,0.8]

In [ ]:
#assign each value based on function
df["weekdayfactor"]=np.select(conditions,
                              weekday_factor,
                              default=0)
df["weekendfactor"]=np.select(conditions,
                              weekend_factor,default=0)




In [ ]:
df = df.reset_index(drop=True)
df["df_key"] =df.index


In [ ]:
import dask
import dask.dataframe as dd
import gc

In [ ]:
def optimize_df_types(df):
    """Downcast numeric and convert low-cardinality objects to categories."""
    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_integer_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='integer')

        elif pd.api.types.is_float_dtype(col_type):
            df[col] = pd.to_numeric(df[col], downcast='float')

        elif pd.api.types.is_object_dtype(col_type):
            # Safe cardinality check
            unique_vals = df[col].nunique()
            total_vals = len(df[col])

            if unique_vals < 0.05 * total_vals:
                df[col] = df[col].astype('category')

    return df

In [ ]:
CHUNK_SIZE = 100
num_buildings = len(df)

days_df["df_key"] = 1

print("Starting memory-safe batch processing...")

for i in range(0, num_buildings, CHUNK_SIZE):
    # Extract chunk
    buildings_chunk = df.iloc[i:i + CHUNK_SIZE].copy()
    buildings_chunk["df_key"] = 1

    # Cartesian merge
    chunk_aligned = pd.merge(
        buildings_chunk, days_df,
        on="df_key", how="inner"
    ).drop(columns=["df_key"])

    # Memory optimization
    chunk_aligned = optimize_df_types(chunk_aligned)

    # Save as parquet
    file_name = f'processed_chunk_{i}_to_{i + CHUNK_SIZE - 1}.parquet'
    chunk_aligned.to_parquet(file_name, index=False)

    # Cleanup
    del chunk_aligned
    del buildings_chunk
    gc.collect()

    print(f"Processed chunk {i} → {i + CHUNK_SIZE - 1} → saved {file_name}")

print("Done! All chunks processed.")

Starting memory-safe batch processing...
Processed chunk 0 → 99 → saved processed_chunk_0_to_99.parquet
Processed chunk 100 → 199 → saved processed_chunk_100_to_199.parquet
Processed chunk 200 → 299 → saved processed_chunk_200_to_299.parquet
Processed chunk 300 → 399 → saved processed_chunk_300_to_399.parquet
Processed chunk 400 → 499 → saved processed_chunk_400_to_499.parquet
Processed chunk 500 → 599 → saved processed_chunk_500_to_599.parquet
Processed chunk 600 → 699 → saved processed_chunk_600_to_699.parquet
Processed chunk 700 → 799 → saved processed_chunk_700_to_799.parquet
Processed chunk 800 → 899 → saved processed_chunk_800_to_899.parquet
Processed chunk 900 → 999 → saved processed_chunk_900_to_999.parquet
Processed chunk 1000 → 1099 → saved processed_chunk_1000_to_1099.parquet
Processed chunk 1100 → 1199 → saved processed_chunk_1100_to_1199.parquet
Processed chunk 1200 → 1299 → saved processed_chunk_1200_to_1299.parquet
Processed chunk 1300 → 1399 → saved processed_chunk_1300

In [ ]:
final_df=dd.read_parquet("processed_chunk_*.parquet")
final_df.head()

,fid,full_id,osm_id,cleanfunct,new AREA,usableroof,irradkimea,irradkW_da,iradkW_yea,energydemand/year,occupancy,weekdayfactor,weekendfactor,date,hour,weekday,isweekend
0,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 00:00:00,0,Monday,False
1,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 01:00:00,1,Monday,False
2,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 02:00:00,2,Monday,False
3,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 03:00:00,3,Monday,False
4,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 04:00:00,4,Monday,False


In [ ]:
final_df.dtypes

,0
fid,int8
full_id,category
osm_id,int32
cleanfunct,category
new AREA,float32
usableroof,float32
irradkimea,float32
irradkW_da,float32
iradkW_yea,float32
energydemand/year,float64


In [ ]:
final_df.shape[0].compute()

28770875

In [ ]:
#assign the factor based on day of the week
final_df["finalfactor"]=final_df["weekendfactor"].where(
                           final_df["isweekend"],
                           final_df["weekdayfactor"])
final_df=final_df.compute()


In [ ]:
final_df.head()

,fid,full_id,osm_id,cleanfunct,new AREA,usableroof,irradkimea,irradkW_da,iradkW_yea,energydemand/year,occupancy,weekdayfactor,weekendfactor,date,hour,weekday,isweekend,finalfactor
0,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 00:00:00,0,Monday,False,1.0
1,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 01:00:00,1,Monday,False,1.0
2,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 02:00:00,2,Monday,False,1.0
3,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 03:00:00,3,Monday,False,1.0
4,1,w48161236,48161236,institution,575.204468,402.643127,3.655858,294.399994,80592.0,120792.9426,115.040901,1.0,1.0,01/01/2024 04:00:00,4,Monday,False,1.0


In [ ]:
#create energydaily demand
final_df["dailyenergydemand"]=final_df["energydemand/year"]/365


In [ ]:
final_df= dd.from_pandas(final_df, npartitions=100)

In [ ]:

alpha = 0.03

#Compute mean irradiance per full_id using apply
def compute_mean_irr(df):
    df['mean_irr'] = df.groupby('full_id')['irradkW_da'].transform('mean')
    df['irr_factor'] = 1 + alpha * ((df['irradkW_da'] - df['mean_irr']) / (df['mean_irr'] + 1e-9))
    return df

final_df=final_df.map_partitions(compute_mean_irr)

# 2️⃣ Add random noise safely
def add_noise(df, scale=0.05):
    df = df.copy()
    df['noise'] = np.random.normal(loc=0, scale=scale, size=len(df))
    return df

final_df=final_df.map_partitions(add_noise, scale=0.05)

# 3️⃣ Compute simulated energy
final_df['simuweekday_energy'] = (final_df['dailyenergydemand'] *
                          final_df['weekdayfactor'] *
                          final_df['irr_factor'] *
                          (1 + final_df['noise']))

final_df['simuweekend_energy'] = (final_df['dailyenergydemand'] *
                          final_df['weekendfactor'] *
                          final_df['irr_factor'] *
                          (1 + final_df['noise']))




/tmp/ipython-input-3392697198.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['mean_irr'] = df.groupby('full_id')['irradkW_da'].transform('mean')


In [ ]:
final_df = final_df.compute()

/tmp/ipython-input-3392697198.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['mean_irr'] = df.groupby('full_id')['irradkW_da'].transform('mean')
/tmp/ipython-input-3392697198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean_irr'] = df.groupby('full_id')['irradkW_da'].transform('mean')
/tmp/ipython-input-3392697198.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
#average daily demand wrt building function
avg_demand = combined.groupby("cleanfunct")["simulatedenergy"].mean().sort_values()
plt.figure(figsize=(8, 5))
plt.bar(avg_demand.index, avg_demand.values)
plt.title('average daily demand wrt building function')


In [ ]:
#how much of demand is met by solar
combined["solar_coverage_%"] = (combined["irradkW_da"] / combined["simulatedenergy"]) * 100

# Difference
combined["surplus_or_deficit"] = combined["irradkW_da"] - combined["simulatedenergy"]
summary = combined[["simulatedenergy","irradkW_da","solar_coverage_%"]].describe()
print(summary)

#correlation between irradiance & energy demand
plt.figure(figsize=(10,6))
sns.scatterplot(x=combined["irradkW_da"],y=combined["simulatedenergy"], hue=combined["cleanfunct"])
plt.title('energydemand versus irradiance(daily)')


#findings
Average solar coverage: ~51% of building energy demand — solar is helpful but usually not enough alone.

Median coverage: 21% — most buildings get limited solar contribution due to small roof areas and low irradiance.

High-end coverage (75th percentile): ~57% — larger or well-situated buildings could benefit from solar + battery systems.

Maximum coverage: 578% — some buildings can generate far more than they consume, enabling microgrids or energy sharing.

In [ ]:
#energy demand distribution wrt to irradiance values
irrad_bins=pd.cut(combined["irradkW_da"],
                   bins=3,
                   labels=['low','medium','high'])
combined["irrad_type"]= irrad_bins
plt.figure(figsize=(8,8))
sns.boxplot(x=combined["irrad_type"], y=combined["simulatedenergy"])
plt.title('energy demand distribution wrt to irradiance')

the higher the irradiance level, the higher the median energy demand and the greater the variation in that demand.


In [ ]:
#encoding building function
#commercial,institution,residential(order)
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform="pandas")
buildfunc_encoded=encoder.fit_transform(combined[["cleanfunct"]])
combined_df=pd.concat([combined,buildfunc_encoded], axis=1).drop(columns="cleanfunct")



In [ ]:
#sort in chronological order
combined_df=combined_df.sort_values(by="date", ascending=True).reset_index(True)


In [ ]:
#check correlations against energydemand
combined_df2=combined_df.copy()
combined_df2.drop(columns=["fid","usableroof","irradkimea","iradkW_yea","energydemand/year","weekdayfactor",
                  "weekday","isweekend","finalfactor","dailyenergydemand","irrad_type","irrad_type",
                  "weekendfactor","mean_irr","irr_factor","osm_id","date","hour","irradkW_da","surplus_or_deficit","solar_coverage_%"],inplace=True)

#correlations
corr=combined_df2.drop(columns="full_id", axis=1).corr()
corr["simulatedenergy"].sort_values(ascending=False)


In [ ]:
combined_df2.hist(bins='auto', figsize=(10,8))
plt.show()


In [ ]:

from sklearn.model_selection import train_test_split

cutoff_date = pd.to_datetime('2024-11-01 23:00:00')

train_df=combined_df[combined_df["dates"]<cutoff_date]
test_df=combined_df[combined_df["dates"]>=cutoff_date]


In [ ]:
# hash split the dataset
import hashlib
def test_set_check(identifier,test_ratio,hash=hashlib.md5):
    return hash(str(identifier).encode('utf-8')).digest()[-1]< 256*test_ratio

In [ ]:
def split_train_test_by_id(data,test_ratio,id_column,hash=hashlib.md5):
    ids = data[id_column]
    # Apply test_set_check to each ID
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    # Return train and test sets
    return data.loc[in_test_set], data.loc[in_test_set]
train_df,test_df= split_train_test_by_id(combined_df2, 0.2, "full_id")

In [ ]:
#assign train test values
train_X,train_y=train_df.drop(columns=["simulatedenergy","full_id","date","irradkW_da"], axis=1),train_df["simulatedenergy"]
test_X,test_y=test_df.drop(columns=["simulatedenergy","full_id","date","irradkW_da"], axis=1),test_df["simulatedenergy"]
# scaling the input features data
from sklearn.preprocessing import RobustScaler
scaler=RobustScaler()
trainX_scaled = scaler.fit_transform(train_X)
testX_scaled=scaler.transform(test_X)





In [ ]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from xgboost import XGBRegressor
import optuna

In [ ]:
#timeseriessplit cross validation

tscv=TimeSeriesSplit(n_splits=5)

trainX_scaled = pd.DataFrame(trainX_scaled, columns=train_X.columns, index=train_X.index)
train_y = pd.Series(train_y)

def objective(trial):

    param = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42
    }

    xgb_model= XGBRegressor(**param,eval_metric='rmse',
               early_stopping_rounds=10,verbosity=0
                            )
    rmse_scores = []


    for fold,(train_index, val_index )in enumerate(tscv.split(trainX_scaled,train_y)):
        X_train_cv, X_val_cv =trainX_scaled.iloc[train_index],trainX_scaled.iloc[val_index]
        y_train_cv, y_val_cv =train_y.iloc[train_index],train_y.iloc[val_index]


        xgb_model.fit(X_train_cv, y_train_cv,
                  eval_set=[(X_val_cv, y_val_cv)]

                  )

        predictions = xgb_model.predict(X_val_cv)
        rmse = np.sqrt(mean_squared_error(y_val_cv,predictions))
        rmse_scores.append(rmse)
    return np.mean(rmse_scores)
    #Run Automatic Tuning (Bayesian Optimization)





In [ ]:
#Run Automatic Tuning (Bayesian Optimization)
study = optuna.create_study(direction='minimize')



In [ ]:
study.optimize(objective, n_trials=50, show_progress_bar=True)


In [ ]:
#  Train Final Model with Best Hyperparameters on FULL Training Data
best_params = study.best_params
best_params['objective'] = 'reg:squarederror'
best_params['random_state'] = 42

final_xgb_model = XGBRegressor(**best_params)
# Fit on the *entire* training set
final_xgb_model.fit(trainX_scaled,train_y)



In [ ]:
#Evaluate model
y_pred_final = final_xgb_model.predict(testX_scaled)

In [ ]:

#Calculate key metrics
mse = mean_squared_error(test_y,y_pred_final)
rmse = np.sqrt(mse) # Root Mean Squared Error
mae = mean_absolute_error(test_y,y_pred_final) # Mean Absolute Error (if you used this as an objective)
r2 = r2_score(test_y, y_pred_final)

print(f"Test RMSE: {rmse}")
print(f"Test MAE: {mae}")
print(f"Test R-squared: {r2}")

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=test_y, y=y_pred_final)
plt.plot([test_y.min(), test_y.max()], [test_y.min(),test_y.max()], 'r--')
plt.xlabel("Actual Energy Demand")
plt.ylabel("Predicted Energy Demand")
plt.title("Actual vs Predicted Energy Demand")

In [ ]:
#feature importance
xgb.plot_importance(final_xgb_model, max_num_features=10)
plt.show()

In [ ]:
#error analysis
residuals = test_y.values - y_pred_final
plt.figure(figsize=(8,5))
plt.scatter(test_y, residuals, alpha=0.3)
plt.axhline(0, color='red')
plt.xlabel("Actual Energy Demand")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residual Plot")
plt.show()

The residual plot shows clear patterns and increasing variance at higher energy demand values. This indicates the model is missing important explanatory variables and struggles to predict high-demand buildings accurately. The presence of clustered residual bands suggests systematic under- or over-prediction for specific categories of buildings. Overall, the model does not fully capture the underlying structure of the energy demand data.


In [ ]:
category= [ 'cleanfunct_commercial',
    'cleanfunct_residential',
    'cleanfunct_institution'
]
# Determine which column has the value 1
building_category_test =test_X[category].idxmax(axis=1)
df_test = pd.DataFrame({'Actual':test_y, 'Predicted': y_pred_final, 'Category': building_category_test})
category_error = df_test.groupby('Category').apply(lambda x: ((x['Actual'] - x['Predicted'])**2).mean()**0.5)
print(category_error)

Error analysis by building function reveals that the model performs well for residential buildings (mean residual ≈ 0), moderately underpredicts commercial buildings (mean residual ≈ 5), and significantly underpredicts institutional buildings (mean residual ≈ 30). This indicates that institutional buildings exhibit energy consumption patterns that are not fully captured by the current feature set, leading to systematic underprediction. These results explain the clusters and widening residual patterns seen in the residual plot.

In [ ]:
combined_df2

In [ ]:
#prediction for the full dataset
combined_df3=combined_df2.copy()
scaled_full=scaler.transform(combined_df3.drop(columns=["date","simulatedenergy","full_id","Hour","is_weekend","dailyenergy_usage"], axis=1))
full_pred = final_xgb_model.predict(scaled_full)
#add the predicted values to the original dataset
combined_df3["dailyenergy_usage"]=full_pred


In [ ]:
#energy usage wrt to building function

building_category=combined_df3[category].idxmax(axis=1)
combined_df3['building_category'] = building_category.astype('category')
avgdaily_consumption=combined_df3.groupby('building_category')["dailyenergy_usage"].mean().sort_values()

plt.figure(figsize=(10,8))
sns.barplot(x=avgdaily_consumption.index, y=avgdaily_consumption.values)
plt.title('Average Daily Energy Consumption per Building Category(kibera)')
plt.ylabel('Average Daily Energy Consumption (kWh/day)')
plt.xlabel('Building Category')



In [ ]:

# Ensure date column is formatted and add a 'is_weekend' flag
combined_df3['date'] = pd.to_datetime(combined_df3['date'])

combined_df3['is_weekend'] = combined_df3['date'].dt.dayofweek >= 5

# Calculate Average Consumption for Weekdays and Weekends ---
# Filter dataframes
weekday_df = combined_df3[combined_df3['is_weekend'] == False]
weekend_df = combined_df3[combined_df3['is_weekend'] == True]
avg_weekday_consumption = weekday_df.groupby('building_category')["dailyenergy_usage"].mean().sort_values().reset_index()
avg_weekend_consumption = weekend_df.groupby('building_category')["dailyenergy_usage"].mean().sort_values().reset_index()

In [ ]:
# Weekday Consumption wrt to building function
plt.figure(figsize=(10, 6))
sns.barplot(x='building_category', y='dailyenergy_usage', data=avg_weekday_consumption, palette='Blues_d')
plt.title('Average Daily Energy Consumption (kWh) on weekday by Building Category')
plt.ylabel('Average Daily kWh')
plt.xlabel('Building Category')

# Weekend Consumption wrt to building function
plt.figure(figsize=(10, 6))
sns.barplot(x='building_category', y='dailyenergy_usage', data=avg_weekend_consumption, palette='Reds_d')
plt.title('Average Daily Energy Consumption (kWh) on weekends by Building Category')
plt.ylabel('Average Daily kWh')
plt.xlabel('Building Category')



The analysis of predicted daily energy consumption shows a surprisingly uniform usage pattern across weekdays and weekends for all building categories. This indicates a consistent base load throughout the week in the study area. Therefore, the primary justification for focusing mass solar harnessing on Institutional and Commercial areas is based on the economy of scale and the availability of large, efficient rooftop space, rather than temporal load matching.


In [ ]:
plt.figure(figsize=(10, 6))

# the relationship between irradiance wrt to energy consumption
sns.regplot(
    data=combined_df2,
    x='irradkW_da',
    y='dailyenergy_usage',
    scatter_kws={'alpha': 0.3, 's': 10},
    line_kws={"color": "red"}
)

plt.title('Relationship Between Daily Solar Irradiance and Energy Consumption')
plt.xlabel('Daily Total Solar Irradiance (kWh/m² or similar unit)')
plt.ylabel('Daily Energy Consumption (kWh/day)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()